In [1]:
import xgboost as xgb
from sklearn.mixture import GaussianMixture

## load trained algorithms

### load SAVIC-P - CB$\alpha$ XGB classifier 

In [2]:
xgbc_sus_cba = xgb.XGBClassifier()
xgbc_sus_cba.load_model('Output/ML/models/xgbc_sus_cba.json')

### load SAVIC-Q - CB$\alpha$ XGB classifier \& regressor

In [3]:
xgbc_kcba = xgb.XGBClassifier()
xgbc_kcba.load_model('Output/ML/models/xgbc_kcba.json')

xgbr_c1_b1_a1 = xgb.XGBRegressor()
xgbr_c1_b1_a1.load_model('Output/ML/models/xgbr_cba_c1_b1_a1.json')

xgbr_c1_b1_a0 = xgb.XGBRegressor()
xgbr_c1_b1_a0.load_model('Output/ML/models/xgbr_cba_c1_b1_a0.json')

xgbr_c1_b0_a1 = xgb.XGBRegressor()
xgbr_c1_b0_a1.load_model('Output/ML/models/xgbr_cba_c1_b0_a1.json')

xgbr_c1_b0_a0 = xgb.XGBRegressor()
xgbr_c1_b0_a0.load_model('Output/ML/models/xgbr_cba_c1_b0_a0.json')

xgbr_c0_b1_a1 = xgb.XGBRegressor()
xgbr_c0_b1_a1.load_model('Output/ML/models/xgbr_cba_c0_b1_a1.json')

xgbr_c0_b1_a0_k1 = xgb.XGBRegressor()
xgbr_c0_b1_a0_k1.load_model('Output/ML/models/xgbr_cba_c0_b1_a0_k1.json')

xgbr_c0_b1_a0_k0 = xgb.XGBRegressor()
xgbr_c0_b1_a0_k0.load_model('Output/ML/models/xgbr_cba_c0_b1_a0_k0.json')

xgbr_c0_b0_a1 = xgb.XGBRegressor()
xgbr_c0_b0_a1.load_model('Output/ML/models/xgbr_cba_c0_b0_a1.json')



### load SAVIC-C - CB$\alpha$ XGB classifier 

In [4]:
gmm_cba_name = 'Output/ML/models/GMM_CBA/GMM_CBA'
means = np.load(gmm_cba_name + '_means.npy')
covar = np.load(gmm_cba_name + '_covariances.npy')
gmm_cba = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_cba.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_cba.weights_ = np.load(gmm_cba_name + '_weights.npy')
gmm_cba.means_ = means
gmm_cba.covariances_ = covar

## SAVIC-P - core, beam \& $\alpha$

### load SAVIC-P - CB$\alpha$ processing function

In [5]:
def SAVIC_P_CBA(df_cba_):
    df_cba_pre_SP_ = df_cba_.copy()
    df_cba_pre_SP_['unstable'] = xgbc_sus_cba.predict(df_cba_pre_SP_.values).astype('bool')
    df_cba_post_SP_ = df_cba_pre_SP_.copy()
    
    return df_cba_post_SP_

### run SAVIC-P - CB$\alpha$ 

In [6]:
df_cba_post_SP = SAVIC_P_CBA(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CBA_pre_P'))
df_cba_post_SP

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True


## SAVIC-Q - core, beam \& $\alpha$

### load SAVIC-Q - CB$\alpha$ processing function

In [7]:
def SAVIC_Q_CBA(df_cba_):
    
    df_cba_uns_ = df_cba_[df_cba_['unstable']].drop(columns = ['unstable'])
    
    df_cba_uns_pre_SQ_ = df_cba_uns_.copy()
    df_cba_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']] = \
    np.log10(df_cba_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])
    df_cba_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c', \
                                   'tau_b', 'log_alph_b', 'D_b', 'vv_b', \
                                   'tau_a', 'log_alph_a', 'D_a', 'vv_a']]
    
    df_cba_uns_postC_SQ_ = df_cba_uns_pre_SQ_.copy()
    df_cba_uns_postC_SQ_['group'] = xgbc_kcba.predict(df_cba_uns_pre_SQ_.values)
    
    df_cba_uns_preQ_SQ_ = df_cba_uns_postC_SQ_.copy()
    df_cba_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b', 'log_alph_a']] = \
    pow(10., df_cba_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b', 'log_alph_a']]) 
    df_cba_uns_preQ_SQ_.columns = ['beta_par_core', 'alph_c', \
                                   'tau_b', 'alph_b', 'D_b', 'vv_b', \
                                   'tau_a', 'alph_a', 'D_a', 'vv_a', 'group']

    
    df_cba_uns_preQ_SQ_0_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 0].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_1_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 1].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_2_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 2].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_3_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 3].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_4_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 4].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_5_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 5].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_6_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 6].drop(columns = ['group'])
    df_cba_uns_preQ_SQ_7_ = df_cba_uns_preQ_SQ_[df_cba_uns_preQ_SQ_['group'] == 7].drop(columns = ['group'])
    
    
    # regressors
    cba_full_col_list_ = ['beta_par_core', 'alph_c', \
                          'tau_b', 'D_b', 'alph_b', 'vv_b', \
                          'tau_a', 'D_a', 'alph_a', 'vv_a', \
                          'Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']

    # group 0 - C1B1A1
    if df_cba_uns_preQ_SQ_0_.size > 0:
        df_cba_uns_preQ_SQ_0_log_ = df_cba_uns_preQ_SQ_0_.copy()

        df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']])
        df_cba_uns_preQ_SQ_0_log_[['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        xgbr_c1_b1_a1.predict(df_cba_uns_preQ_SQ_0_log_).astype(float)

        df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_0_ = df_cba_uns_preQ_SQ_0_log_.copy()
        df_cba_uns_postQ_SQ_0_ = df_cba_uns_postQ_SQ_0_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_0_ = pd.DataFrame(columns = cba_full_col_list_)



    # group 1 - C1B1A0
    if df_cba_uns_preQ_SQ_1_.size > 0:
        df_cba_uns_preQ_SQ_1_log_ = df_cba_uns_preQ_SQ_1_.copy()

        df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_1_log_[['Pow_core', 'Pow_beam', 'kB_angle']] = \
        xgbr_c1_b1_a0.predict(df_cba_uns_preQ_SQ_1_log_).astype(float)

        df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_beam', 'kB_angle']]

        df_cba_uns_postQ_SQ_1_ = df_cba_uns_preQ_SQ_1_log_.copy()
        df_cba_uns_postQ_SQ_1_['Pow_alpha'] = 0
        df_cba_uns_postQ_SQ_1_ = df_cba_uns_postQ_SQ_1_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_1_ = pd.DataFrame(columns = cba_full_col_list_)

    # group 2 - C1B0A1
    if df_cba_uns_preQ_SQ_2_.size > 0:
        df_cba_uns_preQ_SQ_2_log_ = df_cba_uns_preQ_SQ_2_.copy()

        df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_2_log_[['Pow_core', 'Pow_alpha', 'kB_angle']] = \
        xgbr_c1_b0_a1.predict(df_cba_uns_preQ_SQ_2_log_).astype(float)

        df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_2_ = df_cba_uns_preQ_SQ_2_log_.copy()
        df_cba_uns_postQ_SQ_2_['Pow_beam'] = 0
        df_cba_uns_postQ_SQ_2_ = df_cba_uns_postQ_SQ_2_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_2_ = pd.DataFrame(columns = cba_full_col_list_)

    # group 3 - C1B0A0
    if df_cba_uns_preQ_SQ_3_.size > 0:
        df_cba_uns_preQ_SQ_3_log_ = df_cba_uns_preQ_SQ_3_.copy()

        df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_3_log_[['Pow_core', 'kB_angle']] = \
        xgbr_c1_b0_a0.predict(df_cba_uns_preQ_SQ_3_log_).astype(float)

        df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_core', 'kB_angle']]

        df_cba_uns_postQ_SQ_3_ = df_cba_uns_preQ_SQ_3_log_.copy()
        df_cba_uns_postQ_SQ_3_[['Pow_beam', 'Pow_alpha']] = 0
        df_cba_uns_postQ_SQ_3_ = df_cba_uns_postQ_SQ_3_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_3_ = pd.DataFrame(columns = cba_full_col_list_)

    # group 4 - C0B1A1
    if df_cba_uns_preQ_SQ_4_.size > 0:
        df_cba_uns_preQ_SQ_4_log_ = df_cba_uns_preQ_SQ_4_.copy()

        df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_4_log_[['Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        xgbr_c0_b1_a1.predict(df_cba_uns_preQ_SQ_4_log_).astype(float)

        df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_4_ = df_cba_uns_preQ_SQ_4_log_.copy()
        df_cba_uns_postQ_SQ_4_[['Pow_core']] = 0
        df_cba_uns_postQ_SQ_4_ = df_cba_uns_postQ_SQ_4_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_4_ = pd.DataFrame(columns = cba_full_col_list_)


    # group 5 - C0B1A0k1
    if df_cba_uns_preQ_SQ_5_.size > 0:
        df_cba_uns_preQ_SQ_5_log_ = df_cba_uns_preQ_SQ_5_.copy()

        df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_5_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_a0_k1.predict(df_cba_uns_preQ_SQ_5_log_).astype(float)

        df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']]

        df_cba_uns_postQ_SQ_5_ = df_cba_uns_preQ_SQ_5_log_.copy()
        df_cba_uns_postQ_SQ_5_[['Pow_core', 'Pow_alpha']] = 0
        df_cba_uns_postQ_SQ_5_ = df_cba_uns_postQ_SQ_5_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_5_ = pd.DataFrame(columns = cba_full_col_list_)    

    # group 6 - C0B1A0k0
    if df_cba_uns_preQ_SQ_6_.size > 0:
        df_cba_uns_preQ_SQ_6_log_ = df_cba_uns_preQ_SQ_6_.copy()

        df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_6_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_a0_k0.predict(df_cba_uns_preQ_SQ_6_log_).astype(float)

        df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_6_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_beam', 'kB_angle']]

        df_cba_uns_postQ_SQ_6_ = df_cba_uns_preQ_SQ_6_log_.copy()
        df_cba_uns_postQ_SQ_6_[['Pow_core', 'Pow_alpha']] = 0
        df_cba_uns_postQ_SQ_6_ = df_cba_uns_postQ_SQ_6_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_6_ = pd.DataFrame(columns = cba_full_col_list_)    


    # group 7 - C0B0A1
    if df_cba_uns_preQ_SQ_7_.size > 0:
        df_cba_uns_preQ_SQ_7_log_ = df_cba_uns_preQ_SQ_7_.copy()

        df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a']] = \
        np.log10(df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b', 'alph_a']])

        df_cba_uns_preQ_SQ_7_log_[['Pow_alpha', 'kB_angle']] = \
        xgbr_c0_b0_a1.predict(df_cba_uns_preQ_SQ_7_log_).astype(float)

        df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_alpha', 'kB_angle']] = \
        10**df_cba_uns_preQ_SQ_7_log_[['beta_par_core', 'alph_c', 'alph_b','alph_a', 'Pow_alpha', 'kB_angle']]

        df_cba_uns_postQ_SQ_7_ = df_cba_uns_preQ_SQ_7_log_.copy()
        df_cba_uns_postQ_SQ_7_[['Pow_core', 'Pow_beam']] = 0
        df_cba_uns_postQ_SQ_7_ = df_cba_uns_postQ_SQ_7_[cba_full_col_list_]
    else:
        df_cba_uns_postQ_SQ_7_ = pd.DataFrame(columns = cba_full_col_list_)    


    df_cba_uns_postQ_SQ_ = pd.concat([df_cba_uns_postQ_SQ_0_, df_cba_uns_postQ_SQ_1_, df_cba_uns_postQ_SQ_2_, \
                                      df_cba_uns_postQ_SQ_3_, df_cba_uns_postQ_SQ_4_, df_cba_uns_postQ_SQ_5_, \
                                      df_cba_uns_postQ_SQ_6_,df_cba_uns_postQ_SQ_7_]).sort_index()
    
    df_cba_postQ_SQ_ = df_cba_.copy()

    df_cba_postQ_SQ_['group'] = np.nan
    df_cba_postQ_SQ_.loc[df_cba_uns_preQ_SQ_.index, 'group' ] = \
    df_cba_uns_preQ_SQ_['group']
    df_cba_postQ_SQ_['group'] = df_cba_postQ_SQ_['group']#.astype('Int64')

    df_cba_postQ_SQ_[['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle']] = np.nan
    df_cba_postQ_SQ_.loc[df_cba_uns_postQ_SQ_.index, ['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle'] ] = \
    df_cba_uns_postQ_SQ_[['Pow_core', 'Pow_beam', 'Pow_alpha', 'kB_angle'] ]
    
    return df_cba_postQ_SQ_

### run SAVIC-Q - CB$\alpha$ 

In [8]:
df_cba_post_SQ = SAVIC_Q_CBA(df_cba_post_SP)
df_cba_post_SQ

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954


## SAVIC-C - core, beam \& $\alpha$

### load SAVIC-C - CB$\alpha$ processing function

In [9]:
def SAVIC_C_CBA(df_cba_):
    df_cba_uns_ = df_cba_[df_cba_['unstable']].drop(columns = ['unstable'])
    
    df_cba_uns_pre_SC_ = df_cba_uns_.copy()

    df_cba_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_cba_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_cba_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_cba_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_cba_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_cba_uns_pre_SC_['alph_c']
    df_cba_uns_pre_SC_['uns_core'] = df_cba_uns_pre_SC_['uns_OFH'] | df_cba_uns_pre_SC_['uns_FM'] | df_cba_uns_pre_SC_['uns_Mirror'] | df_cba_uns_pre_SC_['uns_IC']


    df_cba_uns_pre_SC_['beta_par_beam'] = df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['D_b'] / df_cba_uns_pre_SC_['tau_b']
    df_cba_uns_pre_SC_['beta_par_alpha'] = df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['D_a'] / df_cba_uns_pre_SC_['tau_a']

    df_cba_uns_pre_SC_['lambda_FH_beta'] = ( (df_cba_uns_pre_SC_['beta_par_core'] + df_cba_uns_pre_SC_['beta_par_beam'] + df_cba_uns_pre_SC_['beta_par_alpha']) - \
    (df_cba_uns_pre_SC_['beta_par_core'] * df_cba_uns_pre_SC_['alph_c'] + df_cba_uns_pre_SC_['beta_par_beam'] * df_cba_uns_pre_SC_['alph_b'] + \
    df_cba_uns_pre_SC_['beta_par_alpha'] * df_cba_uns_pre_SC_['alph_a']) ) / 2.

    df_cba_uns_pre_SC_['lambda_FH_alpha'] = 4 * df_cba_uns_pre_SC_['D_a'] / (1. + df_cba_uns_pre_SC_['D_b'] + 4 * df_cba_uns_pre_SC_['D_a']) * df_cba_uns_pre_SC_['vv_a']**2.
    df_cba_uns_pre_SC_['lambda_FH_beam'] = df_cba_uns_pre_SC_['D_b'] / (1. + df_cba_uns_pre_SC_['D_b'] + 4 * df_cba_uns_pre_SC_['D_a']) * df_cba_uns_pre_SC_['vv_b']**2.
    df_cba_uns_pre_SC_['lambda_FH'] = df_cba_uns_pre_SC_['lambda_FH_beta'] + df_cba_uns_pre_SC_['lambda_FH_alpha'] + df_cba_uns_pre_SC_['lambda_FH_beam']
    df_cba_uns_pre_SC_['lambda_FH'] = df_cba_uns_pre_SC_['lambda_FH'] > 1.

    df_cba_uns_pre_SC_['Pow_core'][df_cba_uns_pre_SC_['Pow_core'] < 0] = 0
    df_cba_uns_pre_SC_['Pow_beam'][df_cba_uns_pre_SC_['Pow_beam'] < 0] = 0
    df_cba_uns_pre_SC_['Pow_alpha'][df_cba_uns_pre_SC_['Pow_alpha'] < 0] = 0

    column_list_cba_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                        'tau_b', 'alph_b', 'D_b', 'vv_b', \
                        'tau_a', 'D_a', 'alph_a', 'vv_a', 'Pow_core', 'Pow_beam', 'Pow_alpha', \
                        'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_cba_uns_pre_SC_cut_ = df_cba_uns_pre_SC_[column_list_cba_]
    
    df_cba_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_cba_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_cba_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_cba_uns_post_SC_ = df_cba_uns_pre_SC_cut_.copy()
    df_cba_uns_post_SC_['cluster'] = gmm_cba.predict(df_cba_uns_pre_SC_cut_.values)
    
    df_cba_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_cba_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_cba_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_cba_uns_post_SC_['r_cluster'] = np.nan
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 0] = 9
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 1] = 2
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 2] = 1
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 3] = 4
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 4] = 7
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 5] = 5
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 6] = 12
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 7] = 1
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 8] = 11
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 9] = 10
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 10] = 7
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 11] = 6
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 12] = 8
    df_cba_uns_post_SC_['r_cluster'][df_cba_uns_post_SC_['cluster'] == 13] = 3

    df_cba_uns_post_SC_['r_cluster'] = df_cba_uns_post_SC_['r_cluster'].astype(int)
    
    df_cba_uns_post_SC_['ins_type'] = np.nan
    ins_types_cba_ = ['IC (C)', 'IC (C); A unstable', \
                      'IC (B), C unstable', 'IC (B); A unstable', 'IC (B); high B anis', 'IC (B); borderline PFH', \
                      'IC (A)', 'IC (A); C absorbing', \
                      'Oblique Firehose', 'Parallel Firehose', \
                      'FM (B), oblique', 'FM (B), oblique; mirror']
    for cluster, ins_type in zip(np.arange(12), ins_types_cba_):
        df_cba_uns_post_SC_['ins_type'][df_cba_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_cba_post_SC_ = df_cba_.copy()
    df_cba_post_SC_['ins_type'] = np.nan
    df_cba_post_SC_.loc[df_cba_uns_post_SC_.index, ['ins_type']] = df_cba_uns_post_SC_['ins_type']
    
    return df_cba_post_SC_

### run SAVIC-C - CB$\alpha$ 

In [10]:
df_cba_post_SC = SAVIC_C_CBA(df_cba_post_SQ)
df_cba_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945,Parallel Firehose
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523,IC (B); high B anis
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767,Oblique Firehose
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413,IC (C)
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371,Oblique Firehose
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954,IC (A); C absorbing


## SAVIC - CB$\alpha$

### load SAVIC - CB$\alpha$ processing function

In [11]:
def SAVIC_CoreBeamAlpha(df_cba_):
    
    df_cba_post_SP_ = SAVIC_P_CBA(df_cba_)
    df_cba_post_SQ_ = SAVIC_Q_CBA(df_cba_post_SP_)
    df_cba_post_SC_ = SAVIC_C_CBA(df_cba_post_SQ_)
    
    return df_cba_post_SC_

### run SAVIC - CB$\alpha$

In [12]:
df_cba_post_SC = SAVIC_CoreBeamAlpha(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CBA_pre_P'))
df_cba_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,tau_a,alph_a,D_a,vv_a,unstable,group,Pow_core,Pow_beam,Pow_alpha,kB_angle,ins_type
0,1.7,0.5,1.0,1.0,0.05,0.5,0.5,0.7,0.02,0.07,True,3.0,0.004833,0.000000,0.000000,0.001945,Parallel Firehose
1,0.6,1.0,0.8,1.0,0.05,0.5,0.4,1.9,0.03,0.06,True,7.0,0.000000,0.000000,0.005254,0.004523,IC (B); high B anis
2,0.5,0.7,1.0,3.5,0.10,1.5,0.8,0.8,0.01,0.00,True,6.0,0.000000,0.226850,0.000000,0.004767,Oblique Firehose
3,2.2,0.8,1.0,0.7,0.10,0.9,0.6,1.0,0.02,0.15,False,NaN,NaN,NaN,NaN,NaN,NaN
4,3.6,0.4,1.2,0.7,0.10,0.8,0.6,1.0,0.03,0.13,True,1.0,0.008698,0.011312,0.000000,0.273413,IC (C)
5,1.0,1.0,1.0,1.2,0.05,1.8,0.5,1.2,0.09,0.20,True,6.0,0.000000,0.012197,0.000000,0.001371,Oblique Firehose
6,0.2,2.4,1.0,3.1,0.10,1.5,0.2,0.9,0.03,0.15,True,3.0,0.035672,0.000000,0.000000,0.001954,IC (A); C absorbing
